In [1]:
import torch
import torch.nn as nn
from torchcrf import CRF
import pandas as pd

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
a = torch.rand(32, 8)
b = torch.zeros([32, 8], dtype=torch.float)
b[0, 0] = 1
loss = nn.BCEWithLogitsLoss(reduction='none')(a, b)

In [ ]:
# loss = torch.where(b != 0, loss, torch.tensor([0.]))
loss

In [33]:
loss = torch.where(b != 0, loss, loss*0.5)

In [37]:
loss.shape

torch.Size([32, 8])

In [44]:
num_tags = 5  # number of tags is 5
model = CRF(num_tags)

In [48]:
seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
emissions = torch.randn(seq_length, batch_size, num_tags) #n_seq x batch x logits
tags = torch.tensor([
  [0, 1], [2, 4], [3, 1]
], dtype=torch.long)  #nseq x batch
loss = model(emissions, tags)



In [57]:
a = torch.rand(30, 32, 142)
b = torch.zeros(30, 32, dtype=torch.long)
model = CRF(a.shape[-1])


In [58]:
model(a, b)

tensor(-4717.1206, grad_fn=<SumBackward0>)

In [5]:
list = []
name = ['train', 'test', 'dev']
for i in name:
    for label in open("/home/xps/educate/code/hust/XQA/docs/src/JointIDSF/PhoATIS/word-level" + f'/{i}/label'):
        if label.strip() not in list:
            list.append(label.strip())

In [7]:
len(list)

28

In [9]:
train = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_train.csv", header=None)


In [14]:
a = {'vietanh': 'ngu'}
a.update({'vie anh': 'ocho'})

(array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
        'atis_ground_service', 'atis_airline', 'atis_abbreviation',
        'atis_quantity'], dtype=object),
 4834)

In [15]:
test = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_test.csv", header=None)
test[0].unique(), len(test)

(array(['atis_flight', 'atis_airfare', 'atis_ground_service',
        'atis_airline', 'atis_flight_time', 'atis_quantity',
        'atis_abbreviation', 'atis_aircraft'], dtype=object),
 800)

In [7]:
lengths = {}
with open("/home/xps/educate/code/hust/XQA/data/raw/PhoATIS/word-level/train/seq.in", 'r') as f:
    for line in f.readlines():
        if len(line.strip().split(' ')) not in lengths.keys():
            lengths[len(line.strip().split(' '))] = 1
        else:
            lengths[len(line.strip().split(' '))] += 1 
        if len(line.strip().split(' ')) == 50:
            print(line.strip()) 
    

tôi muốn một chuyến bay khứ_hồi từ đồng_hới đến côn_đảo khởi_hành vào thứ tư ngày 16 tháng 6 đến côn_đảo vào buổi tối khoảng 7 giờ và trở về đồng_hới vào buổi tối ngày hôm sau khoảng 7 giờ hãng hàng_không nào có chuyến bay phù_hợp với tuyến đường đó


In [ ]:
sorted(lengths, key = lambda d: lengths[d])

In [1]:
import os 

In [2]:
os.path.basename('/hoa/hoctro')

'hoctro'

In [1]:
import pandas as pd
os.environ['dir'] = "/home/xps/educate/code/hust/XQA/data"
from utils.preprocess import string2list, get_label
from collections import Counter
from transformers import AutoTokenizer
import torch

data_dir = os.environ['dir']
raw_dir = data_dir + '/data/raw/PhoATIS'
processed_dir = data_dir + '/data/processed/PhoATIS'
qa_processed = data_dir + '/data/processed/QA'
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
train_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/train.csv")

In [3]:
test_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/test.csv")

In [4]:
dev_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/dev.csv")

In [5]:
run = True
for i in range(len(dev_df)):
    q, a, c, t = dev_df.iloc[i][['question', 'start', 'context', 'text']]
    a, t = string2list(a, 'int'), string2list(t)
    for i in range(len(a)):
        if a[i] + 1 in a:
            print(a)

[449, 450, 450, 449]
[449, 450, 450, 449]
[469, 470, 471, 470]
[469, 470, 471, 470]
[469, 470, 471, 470]
[144, 143, 143, 143]
[144, 143, 143, 143]
[144, 143, 143, 143]
[577, 470, 471, 577]
[115, 116, 115, 115]
[115, 116, 115, 115]
[115, 116, 115, 115]
[412, 413, 413, 413]
[67, 68, 68, 67]
[67, 68, 68, 67]
[491, 491, 490, 491]
[474, 474, 475, 474]
[474, 474, 475, 474]
[474, 474, 475, 474]
[304, 297, 298, 297]
[304, 297, 298, 297]
[285, 286, 286, 285]
[285, 286, 286, 285]
[464, 463, 463, 463]
[464, 463, 463, 463]
[464, 463, 463, 463]
[44, 43, 43, 43]
[44, 43, 43, 43]
[44, 43, 43, 43]
[109, 104, 108, 109]
[441, 440, 440, 440]
[441, 440, 440, 440]
[441, 440, 440, 440]
[524, 523, 524, 523]
[524, 523, 524, 523]
[193, 194, 194, 194]
[92, 93, 93, 93]
[189, 190, 190, 189]
[189, 190, 190, 189]
[368, 369, 369, 369]
[882, 879, 883, 882]
[882, 879, 883, 882]
[50, 51, 50, 50]
[50, 51, 50, 50]
[50, 51, 50, 50]
[455, 456, 455, 455]
[455, 456, 455, 455]
[455, 456, 455, 455]
[613, 613, 614, 613]
[613, 6

In [6]:
i = 5
q, a, c, t = dev_df.iloc[i][['question', 'start', 'context', 'text']]
c, t, a, q

('Nổi tiếng với tên gọi Kinh đô ánh sáng, Paris là một trung tâm văn hóa lớn của thế giới và cũng là một trong những thành phố du lịch thu hút nhất. Sự nhộn nhịp, các công trình kiến trúc và không khí nghệ sĩ đã giúp Paris mỗi năm có đến 30 triệu khách nước ngoài. Thành phố còn được xem như kinh đô của thời trang cao cấp với nhiều khu phố xa xỉ cùng các trung tâm thương mại lớn. Là nơi đặt trụ sở chính của các tổ chức quốc tế như OECD, UNESCO... cộng với những hoạt động đa dạng về tài chính, kinh doanh, chính trị và du lịch đã khiến Paris trở thành một trong những trung tâm trung chuyển lớn nhất trên thế giới và được coi như một trong bốn "thành phố toàn cầu" cùng với New York, Luân Đôn và Tokyo.',
 '@@@mỗi năm có đến 30 triệu khách nước ngoài@@@Kinh đô ánh sáng@@@Kinh đô ánh sáng@@@Kinh đô ánh sáng',
 '[221, 22, 22, 22]',
 'Điều gì đã nói lên Paris là thành phố lý tưởng để khách du lịch?')

In [5]:
# def get_label(input, text, start):

#     sequence_ids = input.sequence_ids()
#     idx = 0
#     while sequence_ids[idx] != 1:
#             idx += 1
#     context_start = idx
#     while sequence_ids[idx] == 1:
#             idx += 1
#     context_end = idx - 1
#     offset = input['offset_mapping']
    

#     start_positions, end_positions = [], []
#     start_positions = []
#     end_positions = []
#     start_char = start
#     end_char = start + len(text)
#     offset = input['offset_mapping'][0]
#     if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
#         start_positions.append(0)
#         end_positions.append(0)
#     else:
#         # Otherwise it's the start and end token positions
#         idx = context_start
#         while idx <= context_end and offset[idx][0] <= start_char:
#             idx += 1
#         start_positions.append(idx - 1)
#         idx = context_end
#         while idx >= context_start and offset[idx][1] >= end_char:
#             idx -= 1
#         end_positions.append(idx + 1)

 
#     return torch.tensor(start_positions, dtype=torch.long), torch.tensor(end_positions, dtype=torch.long)

In [5]:
run = True
for i in range(len(train_df)):
    q, a, c, t = train_df.iloc[i][['question', 'start', 'context', 'text']]
    # input = tokenizer(q.strip(), c, return_tensors='pt',
    #             max_length=386,
    #             truncation="only_second",
    #             return_offsets_mapping=True,
    #             padding="max_length",)
    a, t = string2list(a, 'int'), string2list(t)
    start, end = 0, 0
    for i in range(len(a)):
        s, e = get_label(input, t[0], a[0])
        if e < s:
            print("???", s, e)
            run = False
            break
        start += s 
        end += e
    # if start == 0 or end == 0:
    #     q, c, a, t = q, c, a, t
    #     run = False
    #     break
    if not run:
        break
        
    




In [20]:
i = 100
q, a, c, t = dev_df.iloc[i][['question', 'start', 'context', 'text']]
input = tokenizer(q.strip(), c, return_tensors='pt',
            max_length=500,
            truncation="only_second",
            return_offsets_mapping=True,
            padding="max_length",)
a, t = string2list(a, 'int'), string2list(t)
q, c, t[0], a[0], c[a[0]:]

('Đối tượng nào đã ra tay với tờ báo Charlie Hebdo vào năm 2015?',
 'Vào ngày 7 tháng năm 2015, hai kẻ Hồi giáo cực đoan tấn công trụ sở của tờ báo Charlie Hebdo và giết chết mười ba người và vào ngày 9 tháng 1, một tên khủng bố thứ ba, tuyên bố y là một thành viên của ISIS, đã giết chết bốn con tin trong cuộc tấn công vào một cửa hàng tạp hóa của người Do Thái ở Porte de Vincennes. Một loạt các cuộc tuần hành diễn ra ở các thành phố trên toàn nước Pháp vào ngày 10 và 11 tháng 1 năm 2015 để tưởng niệm các nạn nhân của vụ tấn công Charlie Hebdo, vụ nổ súng tại Montrouge, và cuộc khủng hoảng con tin Porte de Vincennes, đồng thời lên tiếng ủng hộ cho tự do ngôn luận, tự do báo chí và chống chủ nghĩa khủng bố. Các quan chức Pháp ước tính rằng các cuộc tuần hành đã có sự tham gia của khoảng bốn triệu người dân cả nước, trở thành đợt tuần hành công cộng lớn nhất tại Pháp kể từ năm 1944, khi Paris được giải phóng khỏi tay Đức Quốc xã vào cuối cuộc Chiến tranh thế giới thứ hai.',
 'hai kẻ Hồi 

In [35]:
def char2idx(start, end, context):
    subtext = context[start:end+1]
    subtext2 = context[start:end+2]

    for i in range(len(context.split())):
        if context.split()[i] == subtext.split()[0] and context.split()[i+len(subtext.split())-1] == subtext.split()[-1]:
            return i, i+len(subtext.split()) - 1
        elif context.split()[i] == subtext2.split()[0] and context.split()[i+len(subtext2.split())-1] == subtext2.split()[-1]:
            return i, i+len(subtext2.split()) - 1 
def offset2length(offset_map, context=None):
    word_lengths = []
    length = 1
    debug = []
    offset_map = [i for i in offset_map if i.sum() != 0]
    
    for idx in range(1, len(offset_map)-1):
        word = ""
        if offset_map[idx][1] == offset_map[idx+1][0]:
            length += 1
            word += context[offset_map[idx][0]:offset_map[idx][1]]
            print(word)
        else:
            debug += " " + word
            word_lengths.append(length)
            word = ""
            length = 1
    # return word_lengths
    return debug
def QA_metrics(start, end, start_idx, end_idx, input, tokenizer):
    '''
    EM and F1 score for text output
    start = b x n
    '''
    EM = 0
    F1 = 0
    for i in range(start.shape[0]):
        pred = tokenizer.decode(input.input_ids[0][start[i]:end[i]+1])
        true = tokenizer.decode(input.input_ids[0][start_idx[i]:end_idx[i]+1])
        if pred == true:
            EM += 1 
        sum = 0
        text = pred if len(pred.split()) < len(true.split()) else true
        for i in range(len(text.split())):
            if pred.split()[i] == true.split()[i]:
                sum += 1 
        precision = sum/len(pred.split())
        recall = sum/len(true.split())
        F1 += 2/(1/precision + 1/recall)
    return EM/start.shape[0], F1/start.shape[0]
    


def get_label(input, text, start, reverse=False, max_length=300, context=None, question=None):
    '''
    we can make use of original sequence or sub_word sequence (tokenized for labelling)
    - non-reverse: use the mapping to map start and end idx
    - reverse: og start and end idx, and word lengths for mapping, use context for get start and end
    '''

    if reverse:
        start_positions, end_positions = start, start + len(text) - 1
        start_positions, end_positions = char2idx(start_positions, end_positions, context)
        offset_mapping = input.offset_mapping[0]
        word_lengths = offset2length(offset_mapping)
        # while len(word_lengths) < max_length:
        #     word_lengths.append(1)
        return torch.tensor(start_positions, dtype=torch.long), torch.tensor(end_positions, dtype=torch.long), torch.tensor(word_lengths, dtype=torch.long)

        



        

        
        



    sequence_ids = input.sequence_ids()
    idx = 0
    while sequence_ids[idx] != 1:
            idx += 1
    context_start = idx
    while sequence_ids[idx] == 1 and idx < len(sequence_ids) - 1:
            idx += 1
    context_end = idx - 1
    offset = input['offset_mapping']
    

    start_positions, end_positions = 0, 0
    start_char = start
    end_char = start + len(text)
    offset = input['offset_mapping'][0]
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions = 0
        end_positions = 0
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions = idx - 1
        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions = idx + 1


 
    return torch.tensor(start_positions, dtype=torch.long), torch.tensor(end_positions, dtype=torch.long)

In [36]:
# char2idx(a[0], a[0]+len(t[0])-1, c), c[a[0]:a[0]+len(t[0])]
# s, e, w = get_label(input, t[0], a[0], reverse=True, context=c, question=q)
# len(w), len((q + " " + c).split())
offset2length(input.offset_mapping[0], context=c)

à
n
g
à
 
t
n
n
 2
5
,
a
k
ẻ
cự
c
a
n
ấ
n
ông 
V
à
ng
à
th
án
n
ă
2015
ha
H
ồ
g
i
á
c
ự
đ
t
ấ
c
ô
t
r
c
ủ
t
b
á
He
b
v
g
i
ế
ch
ế
m
ư
ờ
b
ng
ư
ờ
v
v
à
ng
à
th
án
1
m
ộ
t
ê
k
h
ủ
b
th
b
a
t
uy
ê
b
l
m
ộ
th
à
v
i
ê
c
ủ
IS
IS
đ
g
i
ế
ch
ế
b
ố
t
cu
ộ
t
ấ
c
ô
v
à
m
ộ
c
ử
h
à
t
ạ
h
ó
c
ủ
ng
ư
ờ
T
h
á
Port
Vince
nnes
M
ộ
lo
ạ
c
á
cu
ộ
t
u
ầ
h
à
di
ễ
r
c
á
th
à
p
h
t
r
ê
to
à
n
ư
ớ
Ph
á
v
à
ng
à
v
th
án
n
ă
đ
ni
ệ
c
á
n
ạ
n
h
c
ủ
v
t
ấ
c
ô
He
b
do
v
n
s
ún
t
ạ
Mont
rou
ge
v
cu
ộ
k
h
ủ
ho
ả
Port
Vince
nnes
đ
ồ
th
ờ
l
ê
t
i
ế
ủ
h
ch
t
ng
ô
l
u
ậ
n
t
b
á
ch
v
ch
ố
ch
ng
h
ĩ
k
h
ủ
b
ố
C
á
q
ch
ứ
Ph
á
ư
ớ
t
ín
c
á
cu
ộ
t
u
ầ
h
à
đ
c
s
th
g
c
ủ
k
ho
ả
b
ố
t
ri
ệ
ng
ư
ờ
d
c
n
ư
ớ
c
th
à
đ
ợ
t
u
ầ
h
à
c
ô


'                                                                                                                                                                                           '